In [ ]:
pip install torch torchaudio soundfile TorchCodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.0 MB/s eta 0:00:00


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torchcodec


In [ ]:
class SingleFileDataset(torch.utils.data.Dataset):
    def __init__(self, file_path, sample_rate=16000, segment_duration=1.0):
        self.sample_rate = sample_rate
        self.num_samples = int(sample_rate * segment_duration)

        wav, sr = torchaudio.load(file_path)

        if wav.shape[0] > 1:
            wav = wav.mean(dim=0, keepdim=True)

        if sr != sample_rate:
            wav = torchaudio.functional.resample(wav, sr, sample_rate)

        self.segments = []
        for start in range(0, wav.shape[1], self.num_samples):
            seg = wav[:, start:start + self.num_samples]
            if seg.shape[1] < self.num_samples:
                seg = F.pad(seg, (0, self.num_samples - seg.shape[1]))
            self.segments.append(seg)

    def __len__(self):
        return len(self.segments)

    def __getitem__(self, idx):
        return self.segments[idx]


In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(1, 64, 7, stride=2, padding=3),
            nn.ReLU(),
            nn.Conv1d(64, 128, 7, stride=2, padding=3),
            nn.ReLU(),
            nn.Conv1d(128, 16, 7, stride=2, padding=3),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
class VectorQuantizer(nn.Module):
    def __init__(self, num_codes=512, code_dim=16, beta=0.25):
        super().__init__()
        self.code_dim = code_dim
        self.num_codes = num_codes
        self.beta = beta

        self.codebook = nn.Embedding(num_codes, code_dim)
        self.codebook.weight.data.uniform_(-1 / num_codes, 1 / num_codes)

    def forward(self, z):
        # z: (B, C, T) → (B*T, C)
        z_perm = z.permute(0, 2, 1).contiguous()
        flat_z = z_perm.view(-1, self.code_dim)

        distances = (
            flat_z.pow(2).sum(1, keepdim=True)
            - 2 * flat_z @ self.codebook.weight.t()
            + self.codebook.weight.pow(2).sum(1)
        )

        indices = torch.argmin(distances, dim=1)
        z_q = self.codebook(indices).view(z_perm.shape)

        # losses
        commit_loss = F.mse_loss(z_q.detach(), z_perm)
        codebook_loss = F.mse_loss(z_q, z_perm.detach())
        loss = codebook_loss + self.beta * commit_loss

        # straight-through estimator
        z_q = z_perm + (z_q - z_perm).detach()
        z_q = z_q.permute(0, 2, 1)

        return z_q, loss, indices


In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose1d(16, 128, 7, stride=2, padding=3, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(128, 64, 7, stride=2, padding=3, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(64, 1, 7, stride=2, padding=3, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
class VQSpeechCodec(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.vq = VectorQuantizer()
        self.decoder = Decoder()

    def forward(self, x):
        z = self.encoder(x)
        z_q, vq_loss, _ = self.vq(z)
        x_hat = self.decoder(z_q)
        return x_hat, vq_loss


In [ ]:
audio_path = "/content/tts_output_2.wav"
device = "cuda" if torch.cuda.is_available() else "cpu"

dataset = SingleFileDataset(audio_path)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

model = VQSpeechCodec().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)

recon_loss_fn = nn.L1Loss()


In [ ]:
epochs = 200

for epoch in range(epochs):
    total_loss = 0
    for seg in loader:
        seg = seg.to(device)

        recon, vq_loss = model(seg)
        recon_loss = recon_loss_fn(recon, seg)

        loss = recon_loss + vq_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(loader):.4f}")


Epoch 1/200 | Loss: 0.1494
Epoch 2/200 | Loss: 0.1337
Epoch 3/200 | Loss: 0.1199
Epoch 4/200 | Loss: 0.1066
Epoch 5/200 | Loss: 0.0951
Epoch 6/200 | Loss: 0.0857
Epoch 7/200 | Loss: 0.0775
Epoch 8/200 | Loss: 0.0722
Epoch 9/200 | Loss: 0.0708
Epoch 10/200 | Loss: 0.0731
Epoch 11/200 | Loss: 0.0747
Epoch 12/200 | Loss: 0.0726
Epoch 13/200 | Loss: 0.0678
Epoch 14/200 | Loss: 0.0627
Epoch 15/200 | Loss: 0.0582
Epoch 16/200 | Loss: 0.0541
Epoch 17/200 | Loss: 0.0511
Epoch 18/200 | Loss: 0.0499
Epoch 19/200 | Loss: 0.0499
Epoch 20/200 | Loss: 0.0502
Epoch 21/200 | Loss: 0.0502
Epoch 22/200 | Loss: 0.0501
Epoch 23/200 | Loss: 0.0500
Epoch 24/200 | Loss: 0.0497
Epoch 25/200 | Loss: 0.0496
Epoch 26/200 | Loss: 0.0498
Epoch 27/200 | Loss: 0.0497
Epoch 28/200 | Loss: 0.0491
Epoch 29/200 | Loss: 0.0486
Epoch 30/200 | Loss: 0.0486
Epoch 31/200 | Loss: 0.0486
Epoch 32/200 | Loss: 0.0486
Epoch 33/200 | Loss: 0.0485
Epoch 34/200 | Loss: 0.0485
Epoch 35/200 | Loss: 0.0487
Epoch 36/200 | Loss: 0.0487
E

In [ ]:
os.makedirs("outputs_vq", exist_ok=True)

model.eval()
with torch.no_grad():
    for i, seg in enumerate(dataset):
        seg = seg.unsqueeze(0).to(device)
        recon, _ = model(seg)

        torchaudio.save(f"outputs_vq/original_{i}.wav", seg.cpu()[0], 16000)
        torchaudio.save(f"outputs_vq/reconstructed_{i}.wav", recon.cpu()[0], 16000)


In [ ]:
from IPython.display import Audio
import os

output_dir = "/content/outputs_vq"

for filename in os.listdir(output_dir):
    if filename.endswith(".wav"):
        file_path = os.path.join(output_dir, filename)
        print(f"Playing: {filename}")
        display(Audio(file_path))

Playing: original_0.wav


Playing: reconstructed_0.wav


Playing: original_1.wav


Playing: reconstructed_1.wav
